### Load data

In [ ]:
import pandas as pd

# Form a dataframe
df = pd.read_excel("../data/Indonesia_data.xlsx", sheet_name="2018-2023'data")
df.columns = df.iloc[0]  # Assign first row values as column names
df = df[1:].reset_index(drop=True)  # Drop the first row and reset index
df["Sector"], df["Company Name"] = df["Company Name"], df["Sector"]
# df = df.sort_values(by=["Company Name", "Year"], ascending=[True, False])

# Assign mean values to fundamental factors based on year
fund_col_start = "CRNCY_ADJ_MKT_CAP"
start_idx = df.columns.get_loc(fund_col_start)
fund_cols = df.columns[start_idx:].tolist()
df[fund_cols] = df.groupby("Year")[fund_cols].transform("mean")
df

,Ticker,Sector,Company Name,Year,TOBIN_Q_RATIO,Inflation,Unemployment,GDP_GROWTH,PMI,ESG_SCORE,...,BOARD_COMPOSITION_SCORE,EXECUTIVE_COMPENSATION_SCORE,SHAREHOLDER_RIGHTS_THEME_SCR,AUDIT_THEME_SCORE,CRNCY_ADJ_MKT_CAP,TOT_DEBT_TO_TOT_EQY,BS_TOT_ASSET,NET_DEBT_EBITDA_ADJUSTED,RETURN_ON_ASSET,RETURN_COM_EQY
0,ADMR IJ Equity,Materials,ADARO MINERALS INDONESIA TBK,2023,2.554738,2.61,5.32,5.048,52.2,2.5,...,4.036564,1.546966,6.332036,4.248312,11524.518852,52.610653,14520.380619,1.336346,5.181957,17.000374
1,ADRO IJ Equity,Energy,ALAMTRI RESOURCES INDONESIA,2023,0.809254,2.61,5.32,5.048,52.2,5.09,...,2.7504,1.331532,5.746394,5.019246,11524.518852,52.610653,14520.380619,1.336346,5.181957,17.000374
2,AMMN IJ Equity,Materials,AMMAN MINERAL INTERNASIONAL,2023,3.892129,2.61,5.32,5.048,52.2,5.92,...,4.033714,2.451159,5.746394,3.369387,11524.518852,52.610653,14520.380619,1.336346,5.181957,17.000374
3,ASII IJ Equity,Industrials,ASTRA INTERNATIONAL TBK PT,2023,1.06752,2.61,5.32,5.048,52.2,3.83,...,5.036723,2.562358,6.332036,6.745622,11524.518852,52.610653,14520.380619,1.336346,5.181957,17.000374
4,BBCA IJ Equity,Financials,BANK CENTRAL ASIA TBK PT,2023,1.650824,2.61,5.32,5.048,52.2,4.79,...,3.112028,3.573153,5.20043,7.885791,11524.518852,52.610653,14520.380619,1.336346,5.181957,17.000374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,TBIG IJ Equity,Communication Services,TOWER BERSAMA INFRASTRUCTURE,2018,1.420994,3.13,5.24,5.174,51.2,1.44,...,2.361878,1.458509,5.746394,2.789154,11522.408778,72.015532,10946.711074,1.549804,7.585018,18.201542
290,NCKL IJ Equity,Materials,TRIMEGAH BANGUN PERSADA TBK,2018,NaN,3.13,5.24,5.174,51.2,NaN,...,NaN,NaN,NaN,NaN,11522.408778,72.015532,10946.711074,1.549804,7.585018,18.201542
291,UNVR IJ Equity,Consumer Staples,UNILEVER INDONESIA TBK PT,2018,17.678335,3.13,5.24,5.174,51.2,3.55,...,5.610576,2.667455,7.136406,4.979922,11522.408778,72.015532,10946.711074,1.549804,7.585018,18.201542
292,UNTR IJ Equity,Energy,UNITED TRACTORS TBK PT,2018,1.414856,3.13,5.24,5.174,51.2,2.88,...,3.508835,0.90302,8.420126,5.013972,11522.408778,72.015532,10946.711074,1.549804,7.585018,18.201542
